# Riemann Hypothesis Validation

This notebook validates the explicit formula for the Riemann Hypothesis using adelic systems.


In [ ]:
import mpmath as mp
import sympy as sp
import sys
import os

# Add utils to path
sys.path.append(os.path.join(os.getcwd(), 'utils'))

# Set precision for faster computation in notebook
mp.mp.dps = 25

print("✅ Libraries loaded successfully")
print(f"mpmath precision: {mp.mp.dps} decimal places")

In [ ]:
# Define Mellin transform function directly in notebook
def mellin_transform(f, s, lim=5.0):
    """Numerical Mellin transform: ∫ f(u) e^{su} du."""
    return mp.quad(lambda u: f(u) * mp.exp(s * u), [-lim, lim], maxdegree=10)

def truncated_gaussian(u, a=5.0, sigma=1.0):
    """Smooth compactly supported Gaussian function."""
    if abs(u) > a:
        return mp.mpf('0')
    return mp.exp(-u**2 / (2 * sigma**2))

# Define test functions as in the main script
def f1(u): 
    return mp.exp(-u**2/2) if abs(u) <= 3 else mp.mpf(0)

def f2(u): 
    return mp.exp(-u**2) if abs(u) <= 2 else mp.mpf(0)

def f3(u): 
    return (1 - u**2/4)**2 if abs(u) <= 2 else mp.mpf(0)

print("✅ Functions defined successfully")
print(f"f1(0) = {f1(0)}")
print(f"f2(0) = {f2(0)}")
print(f"f3(0) = {f3(0)}")
print(f"mellin_transform test: {mellin_transform(f1, mp.mpc(1,0), 2.0)}")

In [ ]:
def prime_sum(f, P=100, K=10):
    """Compute the prime sum using sympy for prime generation"""
    total = mp.mpf('0')
    num_primes = min(int(sp.primepi(P)), 20)  # Limit for notebook execution
    
    print(f"Computing prime sum with {num_primes} primes up to {P}")
    
    for i in range(1, num_primes + 1):
        p = sp.prime(i)
        lp = mp.log(p)
        for k in range(1, K + 1):
            total += lp * f(k * lp)
    return total

def zero_sum(f, max_zeros=10):
    """Compute zero sum using mpmath built-in zeros"""
    total = mp.mpf('0')
    
    print(f"Computing zero sum with {max_zeros} zeros")
    
    for n in range(1, max_zeros + 1):
        rho = mp.zetazero(n)
        gamma = mp.im(rho)
        s = mp.mpc(0.5, gamma)
        contrib = mellin_transform(f, s, 3.0).real
        total += contrib
    
    return total

def simple_archimedean(f):
    """Simplified Archimedean contribution"""
    s_one = mp.mpc(1, 0)
    fhat_one = mellin_transform(f, s_one, 3.0)
    return float(fhat_one.real) * 0.1  # Simplified approximation

print("✅ Validation functions defined")

In [ ]:
# Test the validation for each function
test_functions = {'f1': f1, 'f2': f2, 'f3': f3}
results = []

for name, func in test_functions.items():
    print(f"\n=== Testing {name} ===")
    
    # Compute arithmetic side
    prime_side = prime_sum(func, 50, 5)  # Small values for notebook
    arch_side = simple_archimedean(func)
    arithmetic = prime_side + arch_side
    
    # Compute spectral side
    spectral = zero_sum(func, 5)  # Small values for notebook
    
    # Calculate error
    error = abs(arithmetic - spectral)
    rel_error = error / abs(arithmetic) if abs(arithmetic) > 0 else float('inf')
    
    print(f"Arithmetic side: {arithmetic}")
    print(f"Spectral side:   {spectral}")
    print(f"Absolute error:  {error}")
    print(f"Relative error:  {rel_error}")
    
    results.append({
        'function': name,
        'arithmetic': float(arithmetic),
        'spectral': float(spectral),
        'error': float(error),
        'rel_error': float(rel_error)
    })

In [ ]:
# Summary table
print("\n=== Summary ===")
for result in results:
    print(f"{result['function']}: arithmetic={result['arithmetic']:.6f}, spectral={result['spectral']:.6f}, error={result['error']:.6f}")

print("\n=== Notes ===")
print("- This is a simplified validation using reduced parameters for notebook execution")
print("- The Archimedean term uses a basic approximation")
print("- For full validation, use the command-line script with larger parameters")
print("- The theoretical exact match would require infinite sums and proper Archimedean integration")
print("\n✅ Notebook execution completed successfully!")